### Setup Instructions

If you haven't done so, please follow the [setup instructions](./nlp-tutorial-setup) to prepare your environment.  This tutorial will reference [model.py](https://github.com/outerbounds/tutorials/blob/main/nlp/model.py).

#### What You Will Learn
At the end of this lesson, you will:
    
* Learn how to build a simple NLP model using Tensorflow and scikit-learn to classify fashion reviews
* Learn how to set up your model so that it can be easily used in Metaflow

Now it’s time to build our ML model.  We are going to define our model in a separate file with a custom class called `Nbow_Model`.  The model contains two subcomponents: the count vectorizer for preprocessing and the model.  This class facilitates combining these two components together so that we don't have to deal with them separately.

Here an exaplanation of the various methods in this model:

1. `__init__`: this initializes the count vectorizer, a preprocessor that counts the tokens in the text and a nueral network to do the modeling.
2. `fit`:  when we call `fit`, we first fit the count vectorizer, followed by the model. 
3.  `predict`: similarly, when we call `predict`, we need to transform the data with the count vectorizer before making predictions.
4. `eval_acc`: calculates model accuracy given a dataset and labels
5. `eval_rocauc`: calculates the area under the roc curve given a dataset and labels
6. `model_dict`: This exposes of a dictionary that has two components that form this model, the count vectorizer and the nueral network.  We will use this to serialize the model's data into Metaflow. 
7.  `from_dict`: this allows you to instantiate a NbowModel from a `model_dict` which is useful for de-serializing data in Metaflow.

Anytime you create your own model library or define models in custom classes, we recommend explicitly defining how you will serialize and load the model.  This will minimize the chances that things will break as your model code changes by giving you the ability to make sure any new versions of your code are backward compatible on how to load your model or allow you to deal with serialization/de-serialization accordingly in a way that is transparent to you.  This is the purpose of the `from_dict` method and `model_dict` property.  

For Metaflow, it is very convenient if you have an interface that allows you to save model information that is pickleable, as that is how Metaflow saves data.  That is the purpose of `model_dict` and `from_dict`: they allow saving and retrieving data from a pickleable data structure.

In [ ]:
%%writefile model.py

import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

class NbowModel():
    def __init__(self, vocab_sz):
        self.vocab_sz = vocab_sz
        # Instantiate the CountVectorizer
        self.cv = CountVectorizer(min_df=.005, max_df = .75, stop_words='english', strip_accents='ascii', max_features=self.vocab_sz)
        
        # Define the keras model
        inputs = tf.keras.Input(shape=(self.vocab_sz,), name='Input')
        x = layers.Dropout(0.10)(inputs)
        x = layers.Dense(15, activation="relu", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
        predictions = layers.Dense(1, activation="sigmoid",)(x)
        self.model = tf.keras.Model(inputs, predictions)
        opt = optimizers.Adam(learning_rate=0.002)
        self.model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
        
    def fit(self, X, y):
        res = self.cv.fit_transform(X).toarray()
        self.model.fit(x=res, y=y, batch_size=32, epochs=10, validation_split=.2)
    
    def predict(self, X):
        res = self.cv.transform(X).toarray()
        return self.model.predict(res)
    
    def eval_acc(self, X, labels, threshold=.5):
        return accuracy_score(labels, self.predict(X) > threshold)
    
    def eval_rocauc(self, X, labels):
        return roc_auc_score(labels,  self.predict(X))

    @property
    def model_dict(self): return {'vectorizer':self.cv, 'model': self.model}

    @classmethod
    def from_dict(cls, model_dict):
        "Get Model from dictionary"
        nbow_model = cls(len(model_dict['vectorizer'].vocabulary_))
        nbow_model.model = model_dict['model']
        nbow_model.cv = model_dict['vectorizer']
        return nbow_model

Overwriting model.py


Next, let's import the `NbowModel` and train it on this dataset.  The purpose of doing this is to make sure the code works as we expect before using Metaflow.  For this example, we will set our `vocab_sz = 750`:

In [ ]:
#notest
from model import NbowModel
model = NbowModel(vocab_sz=750)
model.fit(X=df['review'], y=df['labels'])

Epoch 1/10
510/510 [==============================] - 1s 1ms/step - loss: 0.3619 - accuracy: 0.8447 - val_loss: 0.3018 - val_accuracy: 0.8707
Epoch 2/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2945 - accuracy: 0.8797 - val_loss: 0.2923 - val_accuracy: 0.8776
Epoch 3/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2837 - accuracy: 0.8855 - val_loss: 0.2949 - val_accuracy: 0.8763
Epoch 4/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2708 - accuracy: 0.8925 - val_loss: 0.3075 - val_accuracy: 0.8744
Epoch 5/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2623 - accuracy: 0.8982 - val_loss: 0.3036 - val_accuracy: 0.8795
Epoch 6/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2530 - accuracy: 0.9043 - val_loss: 0.3052 - val_accuracy: 0.8756
Epoch 7/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2441 - accuracy: 0.9104 - val_loss: 0.3104 - val_accuracy: 0.8766
Epoch 

Next, we can evaluate our model on the validation set as well, using the built-in evaluation methods we created:

In [ ]:
#notest
model_acc = model.eval_acc(valdf['review'], valdf['labels'])
model_rocauc = model.eval_rocauc(valdf['review'], valdf['labels'])

print(f'Baseline Accuracy: {model_acc:.3f}\nBaseline AUC: {model_rocauc:.3f}')

Baseline Accuracy: 0.875
Baseline AUC: 0.914


Great! This is an improvement upon our baseline!  Now we have set up what we need to start using Metaflow.  In the next lesson, we are going to operationalize the steps we manually performed here into a Flow.